In [1]:

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

import seaborn as sns
sns.set(style="ticks", color_codes=True)

In [2]:
from collections import Counter


from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score, precision_score, recall_score, f1_score

from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score
from sklearn.base import clone

In [3]:
df = pd.read_csv('../Test_Dataset/Training_DataSet.csv', header=0)
df.head()

,ListingID,SellerCity,SellerIsPriv,SellerListSrc,SellerName,SellerRating,SellerRevCnt,SellerState,SellerZip,VehBodystyle,...,VehMake,VehMileage,VehModel,VehPriceLabel,VehSellerNotes,VehType,VehTransmission,VehYear,Vehicle_Trim,Dealer_Listing_Price
0,3287,Warren,False,Inventory Command Center,Prime Motorz,5.0,32,MI,48091.0,SUV,...,Jeep,39319.0,Grand Cherokee,Fair Price,NaN,Used,Automatic 8-Speed,2015,High Altitude,30990.0
1,3920,Fargo,False,Cadillac Certified Program,Gateway Chevrolet Cadillac,4.8,1456,ND,58103.0,SUV,...,Cadillac,30352.0,XT5,Good Deal,Come take a look at our great pre-owned invent...,Used,NaN,2017,NaN,34860.0
2,4777,Waukesha,False,Jeep Certified Program,Wilde Chrysler Jeep Dodge Ram &amp; Subaru,4.8,1405,WI,53186.0,SUV,...,Jeep,38957.0,Grand Cherokee,Good Deal,Backed by a rigorous 125-point inspection by f...,Used,8-Speed Automatic w/OD,2015,Laredo,23249.0
3,6242,Wentzville,False,Inventory Command Center,Century Dodge Chrysler Jeep RAM,4.4,21,MO,63385.0,SUV,...,Jeep,20404.0,Grand Cherokee,Good Deal,Drop by to see us and you will quickly see how...,Used,Automatic,2018,Limited,31977.0
4,7108,Fayetteville,False,HomeNet Automotive,Superior Buick GMC of Fayetteville,3.7,74,AR,72703.0,SUV,...,Cadillac,19788.0,XT5,Good Deal,"Luxury, Exterior Parking Camera Rear, Front Du...",Used,8-Speed Automatic,2018,Luxury,33495.0


In [4]:
df['VehModel'].value_counts()

VehModel
Grand Cherokee    4199
XT5               2099
Name: count, dtype: int64

In [6]:
df_wk2 = df[df['VehModel'] == 'Grand Cherokee']
df_wk2.shape

(4199, 29)

In [7]:
for name in df_wk2.columns:
    vc = df_wk2[name].value_counts()
    if vc.shape[0] < 5:
        print('\n')
        print(vc)




SellerIsPriv
False    4187
True       12
Name: count, dtype: int64


VehBodystyle
SUV    4199
Name: count, dtype: int64


VehCertified
False    3489
True      710
Name: count, dtype: int64


VehFuel
Gasoline         3803
E85 Flex Fuel     343
Diesel             41
Unknown            10
Name: count, dtype: int64


VehMake
Jeep    4199
Name: count, dtype: int64


VehModel
Grand Cherokee    4199
Name: count, dtype: int64


VehPriceLabel
Good Deal     2917
Great Deal     581
Fair Price     503
Name: count, dtype: int64


VehType
Used    4199
Name: count, dtype: int64


In [8]:
# 'ListingID', 
df_wk2.drop(columns=['SellerIsPriv', 'VehType', 'VehBodystyle', 'VehFuel', 'VehMake', 'VehModel'], inplace=True)
df_wk2.head()

/tmp/ipykernel_13605/3112121530.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wk2.drop(columns=['SellerIsPriv', 'VehType', 'VehBodystyle', 'VehFuel', 'VehMake', 'VehModel'], inplace=True)


,ListingID,SellerCity,SellerListSrc,SellerName,SellerRating,SellerRevCnt,SellerState,SellerZip,VehCertified,VehColorExt,...,VehFeats,VehHistory,VehListdays,VehMileage,VehPriceLabel,VehSellerNotes,VehTransmission,VehYear,Vehicle_Trim,Dealer_Listing_Price
0,3287,Warren,Inventory Command Center,Prime Motorz,5.0,32,MI,48091.0,False,White,...,"['Adaptive Cruise Control', 'Antilock Brakes',...","1 Owner, Non-Personal Use Reported, Buyback Pr...",8.600069,39319.0,Fair Price,NaN,Automatic 8-Speed,2015,High Altitude,30990.0
2,4777,Waukesha,Jeep Certified Program,Wilde Chrysler Jeep Dodge Ram &amp; Subaru,4.8,1405,WI,53186.0,True,Brilliant Black Crystal Pearlcoat,...,['18 WHEEL &amp; 8.4 RADIO GROUP-inc: Nav-Capa...,"1 Owner, Buyback Protection Eligible",28.107014,38957.0,Good Deal,Backed by a rigorous 125-point inspection by f...,8-Speed Automatic w/OD,2015,Laredo,23249.0
3,6242,Wentzville,Inventory Command Center,Century Dodge Chrysler Jeep RAM,4.4,21,MO,63385.0,False,Diamond Black Crystal Pearlcoat,...,"['Android Auto', 'Antilock Brakes', 'Apple Car...","1 Owner, Non-Personal Use Reported, Buyback Pr...",59.816875,20404.0,Good Deal,Drop by to see us and you will quickly see how...,Automatic,2018,Limited,31977.0
5,10882,Olean,Digital Motorworks (DMi),Paul Brown Chrysler Dodge Jeep RAM Kia,3.0,51,NY,14760.0,False,Billet Silver Metallic Clearcoat,...,"['1st and 2nd row curtain head airbags', '4-wh...","1 Owner, Non-Personal Use Reported, Buyback Pr...",30.967500,34649.0,Good Deal,Priced below KBB Fair Purchase Price! Clean CA...,8-Speed Automatic,2018,Limited,27900.0
6,12013,Ottawa,Digital Motorworks (DMi),Sierra Motor Mall,3.5,17,IL,61350.0,False,True Blue Pearlcoat,...,"['1st and 2nd row curtain head airbags', '4-wh...","1 Owner, Accident(s) Reported, Non-Personal Us...",194.482338,48814.0,Good Deal,2017 Jeep Grand Cherokee Laredo True Blue Pear...,8-Speed Automatic,2017,Laredo,22989.0


In [9]:
df_wk2.drop(columns=['SellerIsPriv', 'VehTransmission', 'VehEngine'], inplace=True) # XT5
# 'VehTransmission', 'VehEngine' as in US there is only one option
df_wk2.head()

KeyError: "['SellerIsPriv'] not found in axis"

In [13]:
df_wk2.isna().sum(axis=1)>0

0        True
2       False
3       False
5       False
6       False
        ...  
6291    False
6292    False
6293     True
6294    False
6297    False
Length: 4199, dtype: bool

In [11]:
df_wk2['VehSellerNotes'].isna()

0        True
2       False
3       False
5       False
6       False
        ...  
6291    False
6292    False
6293    False
6294    False
6297    False
Name: VehSellerNotes, Length: 4199, dtype: bool

In [ ]:
df_wk2.columns

In [ ]:
cont_cols = ['SellerRating', 'SellerRevCnt', 'VehListdays', 'VehMileage', "Dealer_Listing_Price"]
cat_cols = ['SellerListSrc', 'SellerName', 'VehBodystyle', 'VehCertified', 'VehColorExt', 'VehDriveTrain',
       'VehFeats', 'VehFuel', 'VehHistory', 'VehMake',
       'VehMileage', 'VehPriceLabel', 'VehSellerNotes', 'VehType']
df_wk2[cont_cols] = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(df_wk2[cont_cols])
df_wk2[cat_cols] = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='').fit_transform(df_wk2[cat_cols])
df_wk2["VehYear"] = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=2018).fit_transform(df_wk2[["VehYear"]])
df_wk2["SellerRevCnt"] = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=2018).fit_transform(df_wk2[["SellerRevCnt"]])
# df_wk2["SellerIsPriv"] = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=False).fit_transform(df_wk2[["SellerIsPriv"]])
df_wk2["VehListdays"] = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(df_wk2[["VehListdays"]])

In [ ]:
df_wk2.drop(columns=['VehColorInt'], inplace=True) # nan-cleaning
df_wk2.head()

In [ ]:
nan_indices = df_wk2.isna().sum(axis=1).sort_values(ascending=False)
nan_indices[nan_indices > 0].value_counts()

In [ ]:
nan_indices[nan_indices > 0].shape

In [ ]:
ndf_wk2 = df_wk2[nan_indices == 0]
ndf_wk2.shape

In [ ]:
ndf_wk2.describe()

In [ ]:
ndf_wk2['VehYear'].value_counts()

In [ ]:
ndf_wk2['SellerZip'].unique().shape

In [ ]:
ndf_wk2['VehDriveTrain'].value_counts()

In [ ]:
ndf_wk2['VehHistory'].value_counts()

In [ ]:
history_lst = []
for hist in  ndf_wk2['VehHistory']:
    print(hist, type(hist))
    if isinstance(hist, str):
        history_lst.extend(hist.split(', '))

In [ ]:
Counter(history_lst).most_common(12)

In [ ]:
ndf_wk2['VehPriceLabel'].value_counts()

In [ ]:
ndf_wk2['VehSellerNotes'][:5]

In [ ]:
ndf_wk2.columns

In [ ]:
ndf_wk2['VehYear'].value_counts()

In [ ]:
ndf_wk2['Vehicle_Trim'].value_counts()

In [ ]:
cont_cols = ['SellerRating', 'SellerRevCnt', 'VehListdays', 'VehMileage']
cont_cols

In [ ]:
fig, axs = plt.subplots(
    ncols=len(cont_cols), 
    nrows=len(cont_cols), 
    figsize=(25, 25)
)
for i in range(len(cont_cols)):
    for j in range(i+1):
        if i == j:
            sns.histplot(
                data=ndf_wk2, 
                x=cont_cols[i],
                bins=50, 
                ax=axs[i][i]
                )
        else:
            sns.scatterplot(
                data=ndf_wk2,
                x=cont_cols[i], 
                y=cont_cols[j], 
                hue="Dealer_Listing_Price", 
                # palette='dark',
                legend=True, 
                ax=axs[i][j]
                )

In [ ]:
cont_cols = ['SellerRating', 'SellerRevCnt', 'VehListdays', 'VehMileage', "Dealer_Listing_Price"]
fig, axs = plt.subplots(ncols=len(cont_cols), nrows=len(cont_cols), figsize=(25,25))
for i in range(len(cont_cols)):
    for j in range(i+1):
        if i == j:
            sns.histplot(
                data=ndf_wk2, 
                x=cont_cols[i], 
                hue="VehPriceLabel", 
                palette=sns.color_palette("tab10", 3),  
                bins=50, 
                ax=axs[i][i]
                )
        else:
            sns.scatterplot(
                data=ndf_wk2,
                x=cont_cols[i], 
                y=cont_cols[j], 
                hue="VehPriceLabel", 
                palette=sns.color_palette("tab10", 3),
                legend=True, 
                ax=axs[i][j]
                )

In [ ]:
df_wk2["VehCertified"].value_counts()

In [ ]:
cont_cols = ['SellerRating', 'SellerRevCnt', 'VehListdays', 'VehMileage', "Dealer_Listing_Price"]
fig, axs = plt.subplots(ncols=len(cont_cols), nrows=len(cont_cols), figsize=(25,25))
for i in range(len(cont_cols)):
    for j in range(len(cont_cols)):
        if i == j:
            sns.histplot(
                data=ndf_wk2,
                x=cont_cols[i],
                hue="VehCertified", 
                palette=sns.color_palette("tab10", 2),
                bins=20,
                ax=axs[i][i]
                )
        elif j <i:
            sns.scatterplot(
                data=ndf_wk2,
                x=cont_cols[i], 
                y=cont_cols[j], 
                hue="VehCertified", 
                palette=sns.color_palette("tab10", 2),
                legend=True, 
                ax=axs[i][j]
                )
        else:
            fig.delaxes(axs[i,j])

In [ ]:
ndf_wk2["VehYear"].value_counts()

In [ ]:
cont_cols = ['SellerRating', 'SellerRevCnt', 'VehListdays', 'VehMileage', "Dealer_Listing_Price"]
fig, axs = plt.subplots(ncols=len(cont_cols), nrows=len(cont_cols), figsize=(25,25))
for i in range(len(cont_cols)):
    for j in range(i+1):
        if i == j:
            sns.histplot(
                data=ndf_wk2, 
                x=cont_cols[i], 
                hue="VehYear", 
                palette=sns.color_palette("tab10", 5),  
                bins=50, 
                ax=axs[i][i]
                )
        else:
            sns.scatterplot(
                data=ndf_wk2,
                x=cont_cols[i], 
                y=cont_cols[j], 
                hue="VehYear", 
                palette=sns.color_palette("tab10", 5),
                legend=True, 
                ax=axs[i][j]
                )

In [ ]:
ndf_wk2["VehFuel"].value_counts()

In [ ]:
cont_cols = ['SellerRating', 'SellerRevCnt', 'VehListdays', 'VehMileage', "Dealer_Listing_Price"]
fig, axs = plt.subplots(ncols=len(cont_cols), nrows=len(cont_cols), figsize=(25,25))
for i in range(len(cont_cols)):
    for j in range(len(cont_cols)):
        if i == j:
            sns.histplot(
                data=ndf_wk2, 
                x=cont_cols[i], 
                hue="VehFuel", 
                palette=sns.color_palette("tab10", 4),  
                bins=50, 
                ax=axs[i][i]
                )
        elif j<i:
            sns.scatterplot(
                data=ndf_wk2,
                x=cont_cols[i], 
                y=cont_cols[j], 
                hue="VehFuel", 
                palette=sns.color_palette("tab10", 4),
                legend=True, 
                ax=axs[i][j]
                )
        else:
            fig.delaxes(axs[i][j])

In [ ]:
cont_cols = ['SellerRating', 'SellerRevCnt', 'VehListdays', 'VehMileage', "Dealer_Listing_Price"]
ndf_wk2[cont_cols].corr()

In [ ]:
_ind = np.random.permutation(ndf_wk2.shape[0])
_n = int(0.9 * ndf_wk2.shape[0])
train_index, test_index = _ind[:_n], _ind[_n:]
train_index.shape, test_index.shape

In [ ]:
trim_df = pd.DataFrame()
trim_df['VehFeats'] = ndf_wk2['VehFeats'].map(lambda x: ' '.join([_x[1:-1] for _x in x[1:-1].split(', ')]))
trim_df['VehSellerNotes'] = ndf_wk2['VehSellerNotes']
trim_df['VehColorExt'] = ndf_wk2['VehColorExt']
trim_df['VehDriveTrain'] = ndf_wk2['VehDriveTrain']

trim_df['y'] = OrdinalEncoder().fit_transform(ndf_wk2[['Vehicle_Trim']])
trim_df['y'].value_counts()


In [ ]:
from itertools import chain, combinations
col_names = ['VehFeats', 'VehSellerNotes', 'VehColorExt', 'VehDriveTrain']

for cols in chain.from_iterable(combinations(col_names, r) for r in range(1, len(col_names)+1)):
    print(cols)
    trim_df['data'] = ''
    for col in cols:
        trim_df['data'] += trim_df[col] + ' '

    train_df = trim_df.iloc[train_index]

    count_vector = CountVectorizer()
    tfidf_transformer = TfidfTransformer()
    train_counts = count_vector.fit_transform(train_df['data'])
    train_tfidf = tfidf_transformer.fit_transform(train_counts)

    clf = MultinomialNB().fit(train_tfidf, train_df['y'])

    test_counts = count_vector.transform(trim_df.iloc[test_index]['data'])
    test_tfidf = tfidf_transformer.transform(test_counts)
    y_pred = clf.predict(test_tfidf)

    print(confusion_matrix(y_pred, trim_df.iloc[test_index]['y']))
